# Identifying common elements in two large lists

This interactive example demonstrates that is often important to use the right tool, or computational method, for the task at hand. 

An example of this is the problem I set the first years earlier this year:

Write a function numpy_nested_sum, which takes as input two 1D numpy arrays (x and y) with lengths N and M respectively and computes the following sum:

$\sum_{i=0}^{N-1} \sum_{j=0}^{M-1} \left(x[i]^2 + y[j]^2 \right)$

Many of the students tried something like

In [ ]:
def compute_sum(arrx, arry):
    summ = 0
    for i in range(len(arrx)):
        for j in range(len(arry)):
            summ += arrx[i]**2 + arry[i]**2
    return summ

This directly implements the question as written, but didn't pass the longest test case (due to timeout). Those students that remembered that they were also highly trained mathemeticians did some algebraic reordering and realised that this was equal to:

$\sum_{i=0}^{N-1} (M * x[i]^2) +  \sum_{j=0}^{M-1} ( N * y[j]^2)$

This is then coded as


In [1]:
def compute_sum(arrx, arry):
    summ = 0
    for i in range(len(arrx)):
        summ += M * arrx[i]**2
    
    for j in range(len(arry)):
        summ += N * arry[i]**2
    return summ

and the students got full marks. The hint in the function name (to call this *nested* sum), was a bit unfair of course. However, the principal here is the important one: How many operations do you actually need to do to get the necessary output? This is called "complexity analysis" in computing, this article has a lot more details on this:

https://www.freecodecamp.org/news/big-o-notation-why-it-matters-and-why-it-doesnt-1674cfa8a23c/

Let's consider a different problem:

* Consider two large lists of integers. Identify integers which are in *both* lists

First let's create the lists and then let's take a first stab at this

In [2]:
# Let's create our lists first - DO NOT change this code
import numpy
list_a = numpy.random.randint(0,10000000,size=[100000])
list_b = numpy.random.randint(0,10000000,size=[100000])

In [ ]:
# Then let's write our first solution
def identify_common_elements(list_1, list_2):
    common_elements = []
    # Let's just write a for loop, and for every element check if it's in list_2
    for elem in list_1:
        if elem in list_2:
            common_elements.append(elem)
    return common_elements

# This seems like it should be fast, right??

As before, run timeit and lprof on this code to determine run-time and the slowest line. *Before* running lprof identify where you think the most time will be taken, in which line of the code. Were you right?

In [ ]:
# Run timeit and lprun here


Most of the time is spent determining if elem is in list 2, for every element. There are in total `O(N**2)` operations in this operation, where N is the length of the array. Specifically for each of the `N` elements in list 1 we have to check if any of the `N` elements in list 2 are the same. (There would be some special cases here, for example if the lists have a length of 1000000, but contained only integers between 0 and 10). 

Can we do better? How can we speed up the `elem in list_2` part of the code? How about this:

In [ ]:
# Let's try that again
def identify_common_elements_with_a_set(list_1, list_2):
    # ALL I DO IS ADD THE FOLLOWING 1 LINE
    list_2 = set(list_2)
    common_elements = []
    # Let's just write a for loop, and for every element check if it's in list_2
    for elem in list_1:
        if elem in list_2:
            common_elements.append(elem)
    return common_elements

# This seems like it wouldn't be any faster, right??

Again use timeit and lprof to profile the code

In [ ]:
# Run timeit and lprun here


Think about why that happened?

Sometimes optimisations in python aren't obvious, because a lot of the internal details can be hidden. A `set` and a `list` are similar but very different objects. On the face of it a `set` just contains a list of non-duplicated entries (So `set([1,1,2,3,4])` = `set([1,2,3,4])`). But because non-duplicate entries are not possible the set can check if an item is in the `set`*much* more quickly than you can check if an item is in a list ... Basically python implements a technique called a "hash table" https://en.wikipedia.org/wiki/Hash_table to decide quickly if an object is already in the set. Dictionaries use the same technique with their keys. (Also look up binary search tables as another interesting technique for this kind of thing).

In [ ]:
# One more try
def identify_common_elements_with_two_sets(list_1, list_2):
    list_1 = set(list_1)
    list_2 = set(list_2)
    return list(list_1 & list_2)


Again profile this code, is it quicker than the previous version?

In [ ]:
# Run timeit and lprun here


You can use the `&` operator to take the combination of the two lists. This results in a code with much fewer lines, but the overall cost is much the same. *Converting* a list to a set would be a non-negligible cost here, but if your input is already a set, and your output can be a set then this operation will be faster.:

In [ ]:
set_a = set(list_a)
set_b = set(list_b)
%timeit set_a & set_b

Can we do even better and record integers that occur multiple times in both lists? This algorithm sorts the input and then walks through the sorted arrays looking for duplicates. The sorting is a `N log N` operation, and the walkthrough requires `O(N)` operations.

In [ ]:
# Let's try that again
def identify_common_elements_with_a_walk(list_1, list_2):
    list_1 = sorted(list_1)
    list_2 = sorted(list_2)
    idx_1 = 0
    idx_2 = 0
    l1 = len(list_1)
    l2 = len(list_2)
    common_elements = []
    curr_1 = list_1[idx_1]
    curr_2 = list_2[idx_2]
    while 1:
        if curr_1 == curr_2:
            common_elements.append(curr_1)
            idx_1 += 1
            if idx_1 == l1:
                break
            curr_1 = list_1[idx_1]
            idx_2 += 1
            if idx_2 == l2:
                break
            curr_2 = list_2[idx_2]
        elif curr_1 < curr_2:
            idx_1 += 1
            if idx_1 == l1:
                break
            curr_1 = list_1[idx_1]
        else:
            idx_2 += 1
            if idx_2 == l2:
                break
            curr_2 = list_2[idx_2]
    return common_elements

In [ ]:
# Run timeit and lprun here


For the data arrays we are considering this is not faster, even if the arrays are sorted before sending to the function. That's probably not surprising. The `&` method on the two sets would have implemented this if it were faster!